# Lab 6-1: Softmax Classification

### Imports

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

### Softmax

> - $ P(class = i) = \frac{e^i}{\sum e^i} $

In [27]:
z = torch.FloatTensor([1, 2, 3])
# a = torch.FloatTensor([[1,2],[3,4]])

hypothesis = torch.softmax(z, dim = 0)

hypothesis, torch.sum(hypothesis)

(tensor([0.0900, 0.2447, 0.6652]), tensor(1.))

### Cross Entropy Loss (Low-level)

> - $ L = \frac{1}{N} \sum -y \log( \hat{y} )$

In [37]:
z = torch.rand(3, 5, requires_grad = True)
hypothesis = F.softmax(z, dim = 1)

hypothesis

tensor([[0.2025, 0.2176, 0.2324, 0.1142, 0.2333],
        [0.1503, 0.1465, 0.2203, 0.1962, 0.2866],
        [0.1473, 0.1262, 0.2831, 0.1827, 0.2607]], grad_fn=<SoftmaxBackward>)

In [93]:
y = torch.randint(5, (3,)).long()
y

tensor([0, 1, 2])

In [111]:
y_one_hot = torch.zeros_like(hypothesis)
# y_one_hot.scatter(1, y.unsqueeze(1), 1) # shape, data, num
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # shape, data, num / inplace = True

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [116]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
cost

tensor(1.5931, grad_fn=<MeanBackward0>)

### Cross-entropy  Loss with `torch.nn.functional`

In [118]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.5971, -1.5252, -1.4591, -2.1701, -1.4553],
        [-1.8948, -1.9205, -1.5129, -1.6285, -1.2496],
        [-1.9153, -2.0702, -1.2618, -1.6998, -1.3445]], grad_fn=<LogBackward>)

In [123]:
# High level
# F.softmax() + torch.log() = F.log_softmax()
F.log_softmax(z, dim = 1)

tensor([[-1.5971, -1.5252, -1.4591, -2.1701, -1.4553],
        [-1.8948, -1.9205, -1.5129, -1.6285, -1.2496],
        [-1.9153, -2.0702, -1.2618, -1.6998, -1.3445]],
       grad_fn=<LogSoftmaxBackward>)

In [125]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim = 1))).sum(dim=1).mean()

tensor(1.5931, grad_fn=<MeanBackward0>)

In [128]:
# High level
# negative loss likelihood
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.5931, grad_fn=<NllLossBackward>)

In [129]:
# F.log_softmax() + F.nll_loss() = F.cross_entropy()
F.cross_entropy(z, y)

tensor(1.5931, grad_fn=<NllLossBackward>)

### Training with Low-level Cross Entropy Loss

In [130]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [136]:
# init model
W = torch.zeros((4,3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # H(x)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim = 1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


### Training with `F.cross_entropy`

In [137]:
# init model
W = torch.zeros((4,3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # H(x)
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


### High-level Implementation with `nn.Module`

In [139]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)
        
    def forward(self, x):
        return self.linear(x)
    
model = SoftmaxClassifierModel()

optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # H(x)
    hypothesis = model(x_train)
    
    # cost
    cost = F.cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.465068
Epoch  100/1000 Cost: 0.688907
Epoch  200/1000 Cost: 0.613116
Epoch  300/1000 Cost: 0.557251
Epoch  400/1000 Cost: 0.507291
Epoch  500/1000 Cost: 0.459827
Epoch  600/1000 Cost: 0.413499
Epoch  700/1000 Cost: 0.367648
Epoch  800/1000 Cost: 0.322110
Epoch  900/1000 Cost: 0.278026
Epoch 1000/1000 Cost: 0.245076
